# AgentsVille Trip Planner - Project Assignment

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Project Instructions

Here are the steps you'll follow:

1. **Define Vacation Details**
    - Specify the trip duration, interests, and constraints.
    - Use Pydantic to structure and verify this information in a class called `VacationInfo`.
2. **Review Weather and Activity schedules**
    - Simulate API calls to gather weather data and available activities in bulk
    - Review the data manually to understand the available options
3. **The ItineraryAgent**
    - Implement an agent that generates a day-by-day itinerary based on the vacation details
    - The system prompt will guide the agent's reasoning through a step-by-step planning process to take travel preferences (e.g. destination, dates, interests) and generate a detailed day-by-day itinerary
    - Craft the components of the prompt (including the role, task/instructions, output format, examples, and context) to elicit the best possible itinerary in one LLM call
4. **Evaluating the Itinerary**
    - Evaluate the itinerary using a set of criteria to ensure a high-quality travel plan
        - For instance, does the itinerary match the city and the dates requested?
        - Or, is the total cost calulation accurate and is it within budget?
        - Or, does the agent hallucinate any activities that are not available?
        - Or, does the agent suggest activities that are not suitable for the weather? This specific evaluation function will require the use of an LLM to compare the event description against the weather data.
5. **Defining the Tools**
    - We will define four tools to assist the agent
        - **calculator_tool**: to accurately calculate costs
        - **get_activities_by_date_tool**: to retrieve activities for a specific date
        - **run_evals_tool**: to evaluate the itinerary against the criteria
        - **final_answer_tool**: to provide the final answer in a structured format
6. **The ItineraryRevisionAgent**
    - We will implement a second agent that revises the itinerary based on feedback using the ReAct THOUGHT → ACTION → OBSERVATION cycle.
        - The LLM will first generated a THOUGHT / ACTION message, which contains reasoning steps and a tool call invocation.
        - The Python code will parse the tool call and execute it, returning the result as a string to the LLM in an OBSERVATION message.
        - After this cycle repeats n number of times, the LLM will invoke the final_answer_tool to signal to the Python code to end the loop and return the final answer.
    - This agent will also **incorporate feedback on the initial itinerary** from the travelers to ensure the final plan has **at least 2 activities per day**. A new evaluation function using a powerful LLM will be created to check this user feedback.
    - The agent will use the tools above to refine the plan iteratively, checking the weather and available activities, and ensuring the itinerary meets all constraints.
7. **Something just for fun!**
    - To wrap things up we'll create a fun, narrative summary of the trip, highlighting the best activities and experiences!

## Initial Setup

Let's start with settin up our environment and defining the vacation details.

In [1]:
# When using VSCode in the Udacity workspace, add /workspace to the PYTHON_PATH
import os
import sys

WORKSPACE_DIRECTORY = "/workspace"
if os.path.exists(WORKSPACE_DIRECTORY) and WORKSPACE_DIRECTORY not in sys.path:
    sys.path.append(WORKSPACE_DIRECTORY)
    print(f"Added {WORKSPACE_DIRECTORY} to the Python path")

Added /workspace to the Python path


In [2]:
# Install required packages if not already installed
# No changes needed here.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# If using the Vocareum API endpoint
# TODO: Fill in the missing parts marked with **********
from openai import OpenAI

client = OpenAI(
    # Change the base_url when using the Vocareum API endpoint
    # If using the OpenAI API endpoint, you can comment out the base_url line
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    api_key="******************************",  # <--- TODO: Fill in your Vocareum API key here
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)


In [4]:
# Throughout this project you can experiment with different OpenAI models.
# By default we will use GPT-4.1-mini, which is a good balance of speed and cost.
from enum import Enum

class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning. 
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.

MODEL = OpenAIModel.GPT_41_MINI  # Default model for this project


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In practice, a chatbot agent could gather the information of a user. After it has gathered all the information it needs, it could generate this JSON object from the chat transcript. We will skip that step to focus on the itinerary generation itself.

In [5]:
# The Vacation Info Data Structure
# No changes needed here, but you may choose to personalize the data.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["tennis", "cooking", "comedy", "technology"],
        },
        {
            "name": "Hiro",
            "age": 25,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["reading", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",  # Mock data exists for 2025-06-10
    "date_of_departure": "2025-06-12",  # ...until 2025-06-15.
    "budget": 130,  # Budget is in fictional currency units.
}

In [6]:
# Validate the data structure using Pydantic
# TODO: Fill in the missing parts marked with **********

from project_lib import Interest
from typing import List
from pydantic import BaseModel
import datetime
from pprint import pprint

class Traveler(BaseModel):
    """A traveler with a name, age, and list of interests.
    
    Attributes:
        name (str): The name of the traveler.
        age (int): The age of the traveler.
        interests (List[Interest]): A list of interests of the traveler.
    """
    name: str
    age: int
    interests: List[Interest]

class VacationInfo(BaseModel):
    """Vacation information including travelers, destination, dates, and budget.
    Attributes:
        travelers (List[Traveler]): A list of travelers.
        destination (str): The vacation destination.
        date_of_arrival (datetime.date): The date of arrival.
        date_of_departure (datetime.date): The date of departure.
        budget (int): The budget for the vacation in fictional currency units.
    """
    # TODO: Fill in the the missing fields for the VacationInfo class
    "**********"
    "**********"
    "**********"
    "**********"
    "**********"
    travelers: List[Traveler]
    destination: str
    date_of_arrival: datetime.date
    date_of_departure: datetime.date
    budget: int
    

# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
✅ VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

We will call an API to get all the data at once, in order to be able to include it in the context for our itinerary planning agent.

Also, we will format the data as Pandas DataFrames for easier viewing. Take the time to read and understand the data to see if the agent notices the same things you do. For instance:
- What does the weather look like for the trip? On what days it is sunny, rainy, or cloudy?
- What activities are available on each day? Are there any special events or festivals related to the user's interests?

In [7]:
# The `call_weather_api_mocked` mocks calling a weather API to get weather data
# TODO: Fill in the missing parts marked with **********

from project_lib import call_weather_api_mocked
import pandas as pd

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        # start=**********
        # end=***********
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [8]:
# The `call_activities_api_mocked` function returns the activities for a given date and city.
# TODO: Fill in the missing parts marked with **********

from project_lib import call_activities_api_mocked

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        # start=**********
        # end=***********
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent

First we will review the Pydantic objects used for defining the output of our agent, the TravelPlan, ItineraryDay, Activity, and Weather classes.

Second, we will create a Chain-of-Thought prompt to guide the agent in planning the trip. This prompt will instruct the agent to consider the weather, activities, and user preferences when creating the itinerary.

Third, we will run the agent to produce the TravelPlan object, which will will refine in the following steps.

In [9]:
# Review the data structure we will use for representing a TravelPlan, which includes
# weather, activity_recommendations, and itinerary for each day of the trip.
# Our goal is to take a VacationInfo object and return a TravelPlan object.
# No changes are needed here.

class Weather(BaseModel):
    temperature: float
    temperature_unit: str
    condition: str


class Activity(BaseModel):
    activity_id: str
    name: str
    start_time: datetime.datetime
    end_time: datetime.datetime
    location: str
    description: str
    price: int
    related_interests: List[Interest]


class ActivityRecommendation(BaseModel):
    activity: Activity
    reasons_for_recommendation: List[str]


class ItineraryDay(BaseModel):
    date: datetime.date
    weather: Weather
    activity_recommendations: List[ActivityRecommendation]


class TravelPlan(BaseModel):
    city: str
    start_date: datetime.date
    end_date: datetime.date
    total_cost: int
    itinerary_days: List[ItineraryDay]

In [18]:
# Specify the Chain-of-Thought (CoT) prompt for the Itinerary Agent.
# Remember to include the following:
# [Role]: e.g. Itinerary Planning Agent
# [Task]: Explicitly or implicitly define a step-by-step process for creating the itinerary.
# [Output Format]: Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the specified format.
# [Examples, optional]: Provide examples of how the output should look like.
# [Context]: Make sure to include the weather and activities data in the context, otherwise the agent won't have access to it
# and may instead hallucinate the data.
# TODO: Fill in the missing parts marked with **********

import json 
from project_lib import ChatAgent
from typing import Optional
import json
from project_lib import ChatAgent
from typing import Optional

ITINERARY_AGENT_SYSTEM_PROMPT = f"""
You are the Itinerary Planning Agent for the fictional city of AgentsVille. Your role is to produce accurate, constraint-aware, day-by-day travel plans in JSON format.

## Task
You will receive a VacationInfo object as the user message.
Use it to create a complete TravelPlan that:

- Uses only the activities provided in the Activities context (no invented activities).
- Avoids outdoor-only activities on days with "rainy" or "thunderstorm" weather.
- Selects activities matching traveler interests whenever possible.
- Ensures at least one activity per day.
- Keeps total cost within the provided budget.
- Produces a valid TravelPlan JSON object.

Allowed related_interests values are ONLY:
art, cooking, comedy, dancing, fitness, gardening, hiking, movies, music,
photography, reading, sports, technology, theatre, tennis, writing.

Do NOT create any other values.  
Do NOT create fake activity IDs such as "act1", "act3", "act7", etc.  
Only use activity_id values that appear in the Activities context below.

## Output Format

ANALYSIS:
Explain your reasoning step-by-step:
- Extract constraints from VacationInfo (dates, travelers, interests, budget).
- Interpret weather for each date.
- Filter activities by date, weather, interests.
- Ensure minimum one valid activity per day.
- Ensure total cost stays within budget.
This section MUST NOT contain JSON.

FINAL OUTPUT:
- Provide ONLY a single JSON object matching the TravelPlan schema.
- IMPORTANT:
  - Each activity_recommendation.activity MUST be copied VERBATIM from the Activities context below.
  - NO editing of keys or values.
  - NO changing timestamps.
  - NO modifying related_interests.
  - NO adding or altering activity_id values.

- Wrap the JSON in a ```json code block.

```json
{{
  "city": "<string>",
  "start_date": "<YYYY-MM-DD>",
  "end_date": "<YYYY-MM-DD>",
  "total_cost": <int>,
  "itinerary_days": [
    {{
      "date": "<YYYY-MM-DD>",
      "weather": {{
        "temperature": <float>,
        "temperature_unit": "<string>",
        "condition": "<string>"
      }},
      "activity_recommendations": [
        {{
          "activity": {{
            "activity_id": "<string>",
            "name": "<string>",
            "start_time": "<YYYY-MM-DDTHH:MM:SS>",
            "end_time": "<YYYY-MM-DDTHH:MM:SS>",
            "location": "<string>",
            "description": "<string>",
            "price": <int>,
            "related_interests": ["<string>", "..."]
          }},
          "reasons_for_recommendation": [
            "<string>",
            "..."
          ]
        }}
      ]
    }}
  ]
}}

## Context

Here is the weather forecast for each day of the trip in AgentsVille (list of dictionaries). Use only this data when reasoning about weather:

{json.dumps(weather_for_dates, indent=2)}

Here are the available activities in AgentsVille for each day of the trip (list of dictionaries). You may ONLY choose activities from this list:

{json.dumps(activities_for_dates, indent=2)}

Here is the JSON schema for the TravelPlan object you must produce (use this as a structural guide only; do NOT output this schema in the FINAL OUTPUT):

{json.dumps(TravelPlan.model_json_schema(), indent=2)}
""" # noqa


assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"


class ItineraryAgent(ChatAgent):
    """An agent that plans itineraries based on vacation information, weather, and activities."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates a travel itinerary based on the provided vacation information."""
        from project_lib import print_in_box
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL)


╔══════════════════════════════════════════[ ItineraryAgent - System Prompt ]══════════════════════════════════════════╗
║ You are the Itinerary Planning Agent for the fictional city of AgentsVille. Your role is to produce accurate,        ║
║ constraint-aware, day-by-day travel plans in JSON format.                                                            ║
║ ## Task                                                                                                              ║
║ You will receive a VacationInfo object as the user message.                                                          ║
║ Use it to create a complete TravelPlan that:                                                                         ║
║ - Uses only the activities provided in the Activities context (no invented activities).                              ║
║ - Avoids outdoor-only activities on days with "rainy" or "thunderstorm" weather.                                     ║
║ - Selects activities matching

In [19]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

Now we will create some evaluation functions (sometimes called evals) to help us determine the quality of the itinerary. We will not only want our final output to be of the highest quality possible initially, but we also want to give the chance for the LLM to reflect on its own output and make improvements at a second pass.

If the itinerary does not meet all the criteria specified here, no worries! Afterwards, we will implement a feedback loop that will allow the agent to improve its plan iteratively.

In [20]:
# Helper functions for running the evaluation functions
# No change needed here.

class AgentError(Exception):
    pass


class EvaluationResults(BaseModel):
    success: bool
    failures: List[str]
    eval_functions: List[str]


def get_eval_results(vacation_info, final_output, eval_functions) -> EvaluationResults:
    """
    Evaluates the final output of the itinerary agent against a set of evaluation functions.
    Args:
        vacation_info (VacationInfo): The vacation information used to generate the itinerary.
        final_output (TravelPlan): The final output from the itinerary agent.
        eval_functions (List[callable]): A list of evaluation functions to apply.
    Returns:
        EvaluationResults: An object containing the success status, any failures, and the names of the evaluation functions used.
    """
    from project_lib import print_in_box
    if not isinstance(vacation_info, VacationInfo):
        raise ValueError("vacation_info must be an instance of VacationInfo")
    if not isinstance(final_output, TravelPlan):
        raise ValueError("final_output must be an instance of TravelPlan")
    if not isinstance(eval_functions, list) or not all(
        callable(fn) for fn in eval_functions
    ):
        raise ValueError("eval_functions must be a list of callable functions")
    eval_results = []
    for eval_fn in eval_functions:
        try:
            eval_fn(vacation_info, final_output)
        except AgentError as e:
            error_msg = str(e)
            print_in_box(error_msg, title="Evaluation Error")
            print("\n\n")

            eval_results.append(error_msg)
    return EvaluationResults(
        success=len(eval_results) == 0,
        failures=eval_results,
        eval_functions=[fn.__name__ for fn in eval_functions],
    )


In [21]:
# Basic evaluation functions
# No changes needed here.

def eval_start_end_dates_match(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the arrival and departure dates in vacation_info match the start and end dates in final_output.

    Args:
        vacation_info (dict): Contains the vacation details including arrival and departure dates
        final_output (dict): Contains the itinerary details including start and end dates

    Raises:
        AgentError: If either the arrival date doesn't match the start date or the departure date doesn't match the end date
    """
    if (
        vacation_info.date_of_arrival != final_output.start_date
        or vacation_info.date_of_departure != final_output.end_date
    ):
        raise AgentError(
            f"Dates do not match: {vacation_info.date_of_arrival} != {final_output.start_date} or {vacation_info.date_of_departure} != {final_output.end_date}"
        )

    if final_output.start_date > final_output.end_date:
        raise AgentError(
            f"Start date is after end date: {final_output.start_date} > {final_output.end_date}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_start_end_dates_match],
)

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match'])

In [22]:
# Evaluation functions related to the budget and total cost
# No changes needed here.


def eval_total_cost_is_accurate(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output matches the sum of all activity prices.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including activities with prices and total cost

    Raises:
        AgentError: If the calculated total cost doesn't match the stated total cost
    """
    actual_total_cost = 0

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            actual_total_cost += activity_recommendation.activity.price

    stated_total_cost = int(final_output.total_cost)

    if actual_total_cost != stated_total_cost:
        raise AgentError(
            f"Stated total cost does not match calculated total cost: {actual_total_cost} != {stated_total_cost}"
        )
    
def eval_total_cost_is_within_budget(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output is within the budget specified in vacation_info.

    Args:
        vacation_info (dict): Contains the vacation details including budget
        final_output (dict): Contains the itinerary details including total cost

    Raises:
        AgentError: If the total cost exceeds the budget
    """
    stated_total_cost = int(final_output.total_cost)
    if stated_total_cost > vacation_info.budget:
        raise AgentError(
            f"Total cost exceeds budget: {stated_total_cost} > {vacation_info.budget}"
        )

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_total_cost_is_accurate, eval_total_cost_is_within_budget],
)


EvaluationResults(success=True, failures=[], eval_functions=['eval_total_cost_is_accurate', 'eval_total_cost_is_within_budget'])

In [23]:
# The final output contains copies of the activities, so we need to verify that the copies are accurate
# and not hallucinated.
# No changes needed here.

def eval_itinerary_events_match_actual_events(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the events listed in the itinerary match the actual events

    Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    from project_lib import call_activity_by_id_api_mocked
    event_ids_not_matching = []
    event_ids_missing = []

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            event_id = activity_recommendation.activity.activity_id
            # Assuming get_event_by_id is a function that retrieves the event by its ID

            reference_event = call_activity_by_id_api_mocked(event_id)

            if reference_event is None:
                event_ids_missing.append(event_id)

            elif Activity(**reference_event) != activity_recommendation.activity:
                print(
                    "---\n"
                    f"Event ID {event_id} does not match the reference event:\n"
                    f"Reference Event: {reference_event}\n"
                    f"Activity Event: {activity_recommendation.activity.model_dump()}"
                )
                event_ids_not_matching.append(event_id)
            else:
                # The event matches, so we can continue
                pass

    if event_ids_missing or event_ids_not_matching:
        raise AgentError(
            f"Event IDs missing: {event_ids_missing}\nEvent IDs not matching: {event_ids_not_matching}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_events_match_actual_events],
)


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_events_match_actual_events'])

In [24]:
# Check that the itinerary includes at least one activity matching each traveler's interests.
# No changes needed here.

def eval_itinerary_satisfies_interests(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the itinerary includes activities matching each traveler's interests.

    This function checks that each traveler has at least one activity in the itinerary that matches their interests.

        Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    traveler_to_interests = {}
    traveler_to_interest_hit_counts = {}

    for traveler in vacation_info.travelers:
        traveler_to_interests[traveler.name] = traveler.interests
        traveler_to_interest_hit_counts[traveler.name] = 0

    for traveler_name, interests in traveler_to_interests.items():
        for itinerary_day in final_output.itinerary_days:
            for activity_recommendation in itinerary_day.activity_recommendations:
                # Check if the activity matches any of the traveler's interests
                matching_interests = set(traveler_to_interests[traveler_name]) & set(
                    activity_recommendation.activity.related_interests
                )

                if matching_interests:
                    traveler_to_interest_hit_counts[traveler_name] += 1
                    print(
                        f"✅ Traveler {traveler_name} has a match with interest {matching_interests} at {activity_recommendation.activity.name}"
                    )

    travelers_with_no_interest_hits = [
        traveler
        for traveler, interest_hit_count in traveler_to_interest_hit_counts.items()
        if interest_hit_count == 0
    ]

    # If any of the travelers have 0 matches, raise an error
    if travelers_with_no_interest_hits:
        raise AgentError(
            f"Travelers {travelers_with_no_interest_hits} has no matches with the itinerary."
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)


✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

In [25]:
# Use an LLM to determine whether an event should be avoided due to weather conditions.
# TODO: Fill in the missing parts marked with **********

ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are a classification assistant that decides whether a planned activity is compatible with the expected weather.

## Task
Given:
- the activity name
- a short description
- a single-word weather condition (for example: clear, sunny, partly cloudy, rainy, thunderstorm)

Decide if the activity should be considered compatible with the weather or should be avoided.

Rules:
- Outdoor-only activities (for example, hiking, outdoor picnics, outdoor sports, outdoor concerts) are usually INCOMPATIBLE with "rainy" or "thunderstorm" weather.
- Indoor activities (for example, museum visits, indoor workshops, cooking classes, indoor concerts) remain COMPATIBLE even in bad weather.
- If the description explicitly mentions an indoor backup plan or that the event moves indoors in case of bad weather, treat the activity as COMPATIBLE.
- When there is not enough information to clearly determine whether the activity is indoors or outdoors, assume the activity IS_COMPATIBLE with the weather.
- Focus only on the information given in the description and the weather condition.

## Output format

    REASONING:
    Briefly explain why the activity is compatible or incompatible, referencing the description and the weather.

    FINAL ANSWER:
    IS_COMPATIBLE or IS_INCOMPATIBLE (write exactly one of these two strings).

## Examples

Example 1
Activity: Sunset Ridge Hike
Description: A guided two-hour hike along an exposed mountain trail with panoramic views.
Weather Condition: rainy

REASONING:
The activity is a fully outdoor hike on an exposed trail, and the weather is rainy, which makes hiking unpleasant and potentially unsafe.

FINAL ANSWER:
IS_INCOMPATIBLE


Example 2
Activity: AgentsVille Art Museum Tour
Description: A guided tour through indoor art galleries, entirely inside a climate-controlled museum.
Weather Condition: thunderstorm

REASONING:
The activity takes place completely indoors in a museum, so a thunderstorm outside does not meaningfully impact it.

FINAL ANSWER:
IS_COMPATIBLE


Example 3
Activity: Garden Party with Rain Plan
Description: Outdoor garden party with music and games; if it rains, the event is moved into the adjacent indoor hall.
Weather Condition: rainy

REASONING:
Although the main plan is outdoors, the description clearly states there is an indoor backup plan if it rains, so the event can still go ahead without being ruined by the weather.

FINAL ANSWER:
IS_COMPATIBLE
""".strip()



def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather conditions.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including daily activities and weather conditions

    Raises:
        AgentError: If any outdoor activities are scheduled during weather conditions that could ruin them
    """
    from project_lib import do_chat_completion

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                # This is a high-frequency use case, so we use a fast and cheap model.
                model=OpenAIModel.GPT_41_NANO,
            )
    


            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response from the model: {resp}. Expected 'IS_COMPATIBLE' or 'IS_INCOMPATIBLE'."
                )

            if is_compatible:
                print(
                    f"✅ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are compatible."
                )

            else:
                activities_that_are_incompatible.append(
                    activity_recommendation.activity.name
                )
                print(
                    f"❌ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are incompatible."
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"Activities that may be ruined by inclement weather: {activities_that_are_incompatible}"
        )


eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[
        eval_activities_and_weather_are_compatible
    ],
)

eval_results

✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Experience (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Tech & Film Fusion Night (on 2025-06-12) and weather 'thunderstorm' are compatible.


EvaluationResults(success=True, failures=[], eval_functions=['eval_activities_and_weather_are_compatible'])

In [26]:
# Run all of the evaluation functions again
# No changes needed here.

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()

✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Ex

{'success': True,
 'failures': [],
 'eval_functions': ['eval_start_end_dates_match',
  'eval_total_cost_is_accurate',
  'eval_itinerary_events_match_actual_events',
  'eval_itinerary_satisfies_interests',
  'eval_total_cost_is_within_budget',
  'eval_activities_and_weather_are_compatible']}

## Defining the Tools

Our ItineraryRevisionAgent will be a ReAct-based agent that will use tools to:
- Evaluate/Re-evaluate the itinerary
- Use a calculator since LLMs sometimes struggle with arithmetic
- Call the activities API to get more information about activities
- Return the final itinerary


In [27]:
# Helper function to generate tool descriptions from function docstrings
# No changes needed here.

def get_tool_descriptions_string(fns):
    """Generates a tool description from a function's docstring.
    Args:
        fns (list): List of functions to generate descriptions for.
    Returns:
        str: A formatted string containing the function names and their descriptions."""
    resp = ""
    for fn in fns:
        function_name = fn.__name__
        function_doc = fn.__doc__ or "No description provided."

        resp += f"* `{function_name}`: {function_doc}\n"

    return resp

In [28]:
# Define the calculator tool that evaluates mathematical expressions.
# No changes needed here.

def calculator_tool(input_expression) -> float:
    """Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    """
    import numexpr as ne
    return float(ne.evaluate(input_expression))


assert calculator_tool("1 + 1") == 2.0

print(get_tool_descriptions_string([calculator_tool]))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

Args:
    input_expression (str): A string containing a valid mathematical expression to evaluate.

Returns:
    float: The result of the evaluated expression.

Example:
    >>> calculator_tool("1 + 1")
    2.0




In [29]:
# Tool to fetch activities for a given date and city.
# TODO: Fill in the missing parts marked with **********

def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """Fetches all available activities for a given date and city.

    Args:
        date (str): The date for which to retrieve activities, formatted as 'YYYY-MM-DD'.
        city (str): The name of the city where activities should be searched.

    Returns:
        List[dict]: A list of activity dictionaries, each validated against the Activity model.
                    Each dictionary includes details such as activity_id, name, location, time, 
                    description, price, and related interests.
    """
    from project_lib import call_activities_api_mocked
    resp = call_activities_api_mocked(date=date, city=city)

    return [Activity.model_validate(activity).model_dump() for activity in resp]



assert len(get_activities_by_date_tool("2025-06-10", "AgentsVille")) > 0

print(get_tool_descriptions_string([get_activities_by_date_tool]))

* `get_activities_by_date_tool`: Fetches all available activities for a given date and city.

Args:
    date (str): The date for which to retrieve activities, formatted as 'YYYY-MM-DD'.
    city (str): The name of the city where activities should be searched.

Returns:
    List[dict]: A list of activity dictionaries, each validated against the Activity model.
                Each dictionary includes details such as activity_id, name, location, time, 
                description, price, and related interests.




In [30]:
# Tool to run all evaluation functions on a travel plan.
# No changes needed here.

def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }


print(get_tool_descriptions_string([run_evals_tool]))

* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

Args:
    travel_plan (TravelPlan): The travel plan to evaluate.

Returns:
    EvaluationResults: The results of the evaluations.




In [31]:
# Let's double check that the tool works as expected.
# You should see the same results as before
run_evals_tool(travel_plan=travel_plan_1)

✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Ex

{'success': True, 'failures': []}

In [32]:
# A tool to return the final travel plan
# No changes needed here.

def final_answer_tool(final_output: TravelPlan) -> TravelPlan:
    """Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    """
    return final_output


print(get_tool_descriptions_string([final_answer_tool]))

* `final_answer_tool`: Returns the final travel plan

Args:
    final_output (TravelPlan): The final travel plan to return.

Returns:
    TravelPlan: The final travel plan.




In [33]:
# List of all tools available for the agent
# No changes needed here.

ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

Args:
    input_expression (str): A string containing a valid mathematical expression to evaluate.

Returns:
    float: The result of the evaluated expression.

Example:
    >>> calculator_tool("1 + 1")
    2.0

* `get_activities_by_date_tool`: Fetches all available activities for a given date and city.

Args:
    date (str): The date for which to retrieve activities, formatted as 'YYYY-MM-DD'.
    city (str): The name of the city where activities should be searched.

Returns:
    List[dict]: A list of activity dictionaries, each validated against the Activity model.
                Each dictionary includes details such as activity_id, name, location, time, 
                description, price, and related interests.

* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

Args:
    travel_plan (TravelPlan): The travel plan to evaluate.

Returns:
    Evaluat

## The ItineraryRevisionAgent

The ItineraryRevisionAgent will
* take initial feedback from the user about the itinerary and
* use the tools defined above

to refine original itinerary iteratively using a ReAct-based approach.

In [34]:
# Get the traveler's feedback and create a new evaluation function to check if the feedback was incorporated.
# No changes needed here.

TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan.

    Args:
        vacation_info (VacationInfo): The vacation information.
        final_output (TravelPlan): The revised travel plan.

    Raises:
        AgentError: If the traveler's feedback was not successfully incorporated.
    """

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,  # Use a powerful model for checking traveler feedback
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,  # Add this new evaluation
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Ex

EvaluationResults(success=False, failures=['Traveler feedback was not successfully incorporated into the revised travel plan. Response: PARTIALLY_INCORPORATED\nREASON: The revised travel plan provides at least two activities per day on June 10 and June 11, but only one activity is scheduled for June 12, so the feedback is only partially incorporated.'], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [35]:
# Define the ReAct system prompt for the Itinerary Revision Agent.
# Remember, the output format should include a THOUGHT and a single ACTION (tool call using JSON format).
# Then the Python code will invoke the tool and add an OBSERVATION message to the chat history.
# NOTE: The tool call format should be:
# {"tool_name": "[tool_name]", "arguments": {"arg1": "value1", ...}}

# TODO: Fill in the missing parts marked with **********
from project_lib import print_in_box

ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""
[Role]: Itinerary Revision Agent

[Primary Task]
Revise a provided TravelPlan so it passes the evaluators. Your goal is to return a feasible, evaluator-clean TravelPlan by iteratively inspecting gaps and making precise changes (add, remove, or replace activities).

[THINK–ACT–OBSERVE Loop]
• THINK: Identify the single most important gap to fix (e.g., day has <2 activities, time conflicts, obvious feasibility issues, evaluator feedback).
• ACT: Call exactly ONE tool using the ACTION JSON format (see below).
• OBSERVE: Read the OBSERVATION (tool result), update your mental plan, and repeat.
• EXIT: You MUST call run_evals_tool immediately BEFORE calling final_answer_tool. Only exit via final_answer_tool.

[Available Tools]
1) get_activities_by_date_tool(date: str, city: str) -> List[dict]
   Purpose: Retrieve canonical, cataloged activities (with valid activity_id) for a date and city.
   Params:
     - date: "YYYY-MM-DD"
     - city: string

2) calculator_tool(input_expression: str) -> float
   Purpose: Do arithmetic when you need to total/compare costs or durations.
   Params:
     - input_expression: a valid math expression as a string (e.g., "12.5 + 7.5")

3) run_evals_tool(travel_plan: TravelPlan) -> EvaluationResults
   Purpose: Run the full evaluation suite (e.g., coverage, IDs, feasibility, budget as applicable).
   Required:
     - travel_plan: the current draft TravelPlan JSON you want evaluated.
   Behavior: Returns structured pass/fail details. Use these signals to decide the next fix.

4) final_answer_tool(final_output: TravelPlan) -> TravelPlan
   Purpose: Emit the final, validated TravelPlan. This is the ONLY exit path.

[Action Policy]
• Make minimal, targeted edits that directly address the current gap.
• Use get_activities_by_date_tool to source any new or replacement activities so that activity_id values are canonical.
• Preserve activity_id for retained activities; never fabricate or edit IDs.
• Prefer non-overlapping schedules and at least 2 activities per day.
• Use calculator_tool for simple sums/comparisons if needed (e.g., rough cost arithmetic).

[ACTION JSON – EXACT FORMAT]
SINGLE-CALL RULES:
- Output exactly ONE tool call per cycle.
- Do NOT wrap the ACTION in a list/array.
- Do NOT place the JSON inside code fences.
ACTION:
{{ "tool_name": "[tool_name]", "arguments": {{ "arg1": "value1", "arg2": "value2" }} }}

Examples (format only):
{{ "tool_name": "get_activities_by_date_tool", "arguments": {{ "date": "2025-06-11", "city": "AgentsVille" }} }}
{{ "tool_name": "calculator_tool", "arguments": {{ "input_expression": "12.5 + 7.5" }} }}
{{ "tool_name": "run_evals_tool", "arguments": {{ "travel_plan": TravelPlan }} }}
{{ "tool_name": "final_answer_tool", "arguments": {{ "final_output": TravelPlan }} }}

[Output Structure]
THOUGHT:
- State the specific gap you’re fixing now (e.g., “Coverage shortfall on 2025-06-12” or “Evaluator flagged missing IDs”).
- Briefly justify the next step and name the single tool you will call.

ACTION:
{{ "tool_name": "<one_of_the_tools_above>", "arguments": {{ "<arg1>": "<value1>", "<arg2>": "<value2>" }} }}

[Exit Protocol — REQUIRED SEQUENCE]
1) Immediately before finalizing, you MUST call:
ACTION:
{{ "tool_name": "run_evals_tool", "arguments": {{ "travel_plan": <TravelPlan JSON> }} }}
   - If failures are returned (coverage/IDs/feasibility/budget), fix them iteratively and re-run run_evals_tool.

2) Only when run_evals_tool indicates success, call:
ACTION:
{{ "tool_name": "final_answer_tool", "arguments": {{ "final_output": <TravelPlan JSON> }} }}

[Context — Schemas]

[Context — Schemas]
- VacationInfo:
{VacationInfo.model_json_schema()}
- TravelPlan:
{TravelPlan.model_json_schema()}
- Activity:
{Activity.model_json_schema()}
- Weather:
{Weather.model_json_schema()}
"""  # noqa

class ItineraryRevisionAgent(ChatAgent):
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Extracts the observation from the thought-action response."""

        if "tool_name" not in tool_call_obj:
            return "OBSERVATION: No tool name specified."

        if "arguments" not in tool_call_obj:
            return "OBSERVATION: No arguments specified."

        # If the arguments are not a dictionary, state the error
        if not isinstance(tool_call_obj["arguments"], dict):
            return f"OBSERVATION: Arguments should be a dictionary, got {type(tool_call_obj['arguments'])} instead."

        # If the tool name is not a string, state the error
        if not isinstance(tool_call_obj["tool_name"], str):
            return f"OBSERVATION: Tool name should be a string, got {type(tool_call_obj['tool_name'])} instead."

        tool_name = tool_call_obj["tool_name"]
        arguments = tool_call_obj["arguments"]

        tool_fn = None

        for tool in self.tools:
            if tool.__name__ == tool_name:
                tool_fn = tool
                break

        if tool_fn is None:
            return f"OBSERVATION: Unknown tool name '{tool_name}' in action string."

        try:
            tool_response = tool_fn(**arguments)
            return f"OBSERVATION: Tool {tool_name} called successfully with response: {tool_response}"
        except Exception as e:
            return f"OBSERVATION: Error occurred while calling tool {tool_name}: {e}"

    def run_react_cycle(
        self, original_travel_plan: TravelPlan, max_steps: int = 10, model: Optional[OpenAIModel] = None, client = None,
    ) -> TravelPlan:
        """Runs the ReAct cycle to revise the itinerary based on the evaluation results."""
        from json_repair import repair_json

        # Provide the original travel plan to revise
        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision:\n{original_travel_plan.model_dump_json()}",
        )
        resp = None

        # Run the ReAct cycle for a maximum number of steps
        for step in range(max_steps):
            # Get the thought-action response from the agent
            resp = self.get_response(model=model, client=client) or ""

            # If there is no action, report it to the LLM and continue
            if "ACTION:" not in resp:
                self.add_message(role="user", content="No action found in response.")
                continue

            action_string = resp.split("ACTION:")[1].strip()

            # Parse the tool call JSON from the action string
            try:
                # Fix any JSON formatting issues. e.g. missing closing braces, etc.
                action_string = repair_json(action_string)
                tool_call_obj = json.loads(action_string)
            except json.JSONDecodeError:
                print(f"Invalid JSON in action string: {action_string}")
                self.add_message(
                    role="user",
                    content=f"Invalid JSON in action string: {action_string}",
                )
                continue

            tool_name = tool_call_obj.get("tool_name", None)

            # If the final answer tool is called, validate and return the final travel plan
            if tool_name == "final_answer_tool":
                try:
                    new_travel_plan = TravelPlan.model_validate(
                        tool_call_obj["arguments"].get("final_output", tool_call_obj["arguments"])
                    )
                    return new_travel_plan
                except Exception as e:
                    self.add_message(
                        role="user", content=f"Error validating final answer: {e}"
                    )
                    continue

            # For all other tools, execute the tool call and add the observation
            else:
                # Add the 
                observation_string = self.get_observation_string(
                    tool_call_obj=tool_call_obj
                )
                self.add_message(role="user", content=observation_string)

        raise RuntimeError(
            f"ReAct cycle did not complete within {max_steps} steps. Last response: {resp}"
        )

# Instantiate the Itinerary Revision Agent
itinerary_revision_agent = ItineraryRevisionAgent()

# Let's get a single THOUGHT/ACTION response back to check that the agent is working as expected.
resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision: {travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response")
# Check for THOUGHT
if "THOUGHT:" in resp:
    print("✅ `THOUGHT:` found in raw the response, as expected.")
else:
    print("❌ Expected `THOUGHT:` in raw the response. Please check the system prompt (output format).")
# Check for ACTION
if "ACTION:" in resp:
    print("✅ `ACTION:` found in raw the response, as expected.")
else:
    print("❌ Expected `ACTION:` in raw the response. Please check the system prompt (output format).")
if "\"tool_name\"" in resp:
    print("✅ `\"tool_name\":` found in raw the response, as expected.")
else:
    print("❌ Expected `\"tool_name\":` in raw the response. Please check the system prompt (output format).")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ [Role]: Itinerary Revision Agent                                                                                     ║
║ [Primary Task]                                                                                                       ║
║ Revise a provided TravelPlan so it passes the evaluators. Your goal is to return a feasible, evaluator-clean         ║
║ TravelPlan by iteratively inspecting gaps and making precise changes (add, remove, or replace activities).           ║
║ [THINK–ACT–OBSERVE Loop]                                                                                             ║
║ • THINK: Identify the single most important gap to fix (e.g., day has <2 activities, time conflicts, obvious         ║
║ feasibility issues, evaluator feedback).                                                                             ║
║ • ACT: Call exactly ONE tool 

In [36]:
# Now let's run the ReAct cycle multiple times to get the revised itinerary.
# Note: If this takes more than a few minutes, then the agent may be stuck in a loop.
# Examine the traces to understand where it is failing and see if adjusting the system prompt helps.
# Since LLMs are stochastic, you will get different results each time you run this cell.
# No changes needed here.

itinerary_revision_agent = ItineraryRevisionAgent()
travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1, max_steps=15,
    model=MODEL,
    client=client,
)

print("✅ Revised itinerary generated successfully. Congratulations!")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ [Role]: Itinerary Revision Agent                                                                                     ║
║ [Primary Task]                                                                                                       ║
║ Revise a provided TravelPlan so it passes the evaluators. Your goal is to return a feasible, evaluator-clean         ║
║ TravelPlan by iteratively inspecting gaps and making precise changes (add, remove, or replace activities).           ║
║ [THINK–ACT–OBSERVE Loop]                                                                                             ║
║ • THINK: Identify the single most important gap to fix (e.g., day has <2 activities, time conflicts, obvious         ║
║ feasibility issues, evaluator feedback).                                                                             ║
║ • ACT: Call exactly ONE tool 

In [37]:
# Last let's double check that the revised travel plan passes all evaluation functions.
# No changes needed here.

eval_results_2 = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_2,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

assert eval_results_2.success, f"❌ Read the traces above and modify the system prompt.\n\nFailures: {eval_results_2.failures}"

print("✅ All evaluation functions passed successfully for the revised travel plan.")

eval_results_2

✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Hiro has a match with interest {music} at Soundtrack Picnic: Lunchtime Movies & Melodies
✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [38]:
# Show the final travel plan in a readable format.
# No changes needed here.

from IPython.display import display

for itinerary_day in travel_plan_2.itinerary_days:
    print(f"Date: {itinerary_day.date}")
    print(
        f"Weather: {itinerary_day.weather.condition} ({itinerary_day.weather.temperature}°{itinerary_day.weather.temperature_unit})"
    )

    activities_df = pd.DataFrame(
        [
            activity_recommendation.activity.model_dump()
            for activity_recommendation in itinerary_day.activity_recommendations
        ]
    )
    display(activities_df)

Date: 2025-06-10
Weather: clear (31.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00:00,2025-06-10 11:00:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00:00,2025-06-10 13:30:00,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"


Date: 2025-06-11
Weather: partly cloudy (34.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00:00,2025-06-11 13:30:00,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI creativity).",20,"[technology, music, photography, writing]"
1,event-2025-06-11-3,Palette & Palate: Art Meets Cooking Experience,2025-06-11 18:30:00,2025-06-11 20:30:00,"The Creative Canvas Studio, Artisanal Lane, AgentsVille","Immerse yourself in a colorful evening where art and cooking blend together! At 'Palette & Palate,' participants will begin indoors at The Creative Canvas Studio with a guided session to paint their own culinary-inspired masterpiece. Afterwards, a local chef will lead an interactive cooking class, teaching you how to craft vibrant, edible works of art. Whether you're an art enthusiast, a food lover, or both, this creative night is perfect for socializing and expressing yourself through color and flavor! All materials and ingredients are provided. This event is held indoors and welcomes all experience levels in art and cooking.",25,"[art, cooking]"


Date: 2025-06-12
Weather: thunderstorm (28.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-12-1,Soundtrack Picnic: Lunchtime Movies & Melodies,2025-06-12 12:00:00,2025-06-12 13:30:00,"Starlight Amphitheater, AgentsVille","Experience the magic of classic movie scenes paired with live music at the outdoor Starlight Amphitheater! Bring your lunch and relax on the lawn as musicians perform iconic film soundtracks while selected clips light up our open-air screen. Perfect for movie buffs and music lovers alike, this engaging event celebrates both arts in a sunny lunchtime setting. In case of rain, the event will move indoors to the adjacent Harmony Hall. Come for the tunes, stay for the cinematic wonder!",15,"[movies, music]"
1,event-2025-06-12-3,Tech & Film Fusion Night,2025-06-12 19:00:00,2025-06-12 21:30:00,"Virtual Reality Theater, Silicon Plaza, AgentsVille","Dive into an immersive evening where the magic of movies meets the latest in technology! Join fellow movie buffs and tech enthusiasts for a special screening of cutting-edge sci-fi short films, followed by an interactive panel with local filmmakers and VR technologists. Experience the future of entertainment and discuss how technology is transforming the world of cinema. This exciting, indoor event at the Virtual Reality Theater is perfect for anyone interested in technology and movies.",15,"[technology, movies]"


## And, just for fun!

In [39]:
# And finally, just for fun, let's narrate the trip.
# No changes needed here.

from project_lib import narrate_my_trip

narrate_my_trip(
    vacation_info=vacation_info,
    itinerary=travel_plan_2,
    client=client,
    model=MODEL,  # Optionally, you can change the model here
)


Your upcoming trip to AgentsVille features two travelers, Yuri and Hiro. Yuri is 30 years old with interests in tennis, cooking, comedy, and technology, while Hiro is 25 and enjoys reading, music, theatre, and art. The trip is scheduled from June 10th to June 12th, 2025, with a total budget of 130 units, and the planned itinerary comes in under budget at a total cost of 115 units. The selection of activities carefully reflects your combined interests and takes into account weather conditions to ensure a comfortable and engaging experience.

On June 10th, the weather will be clear with a warm temperature of 31°C, ideal for outdoor activities mixed with indoor events. The day starts with the FutureTech Breakfast Meet-Up at The Innovation Atrium, where Yuri’s passion for technology will be well catered to in a lively setting that encourages networking and discovery of the latest trends. Later, the Serve & Savor: Tennis and Taste Luncheon combines Yuri’s interests in tennis and cooking, offering an energizing mixed experience of playing doubles and participating in a hands-on cooking workshop in an outdoor setting, perfect for enjoying the beautiful weather.

The following day, June 11th, brings partly cloudy skies with a high of 34°C. An indoor event, the Tech Lunch & Learn: AI Frontiers at The Digital Atrium, is perfect for both Yuri and Hiro, blending technology, music, and innovation in an interactive lunchtime gathering. The evening features the Palette & Palate: Art Meets Cooking Experience, a creative and social event where Hiro’s love for art and Yuri’s culinary interests merge. This indoor event offers the chance to paint your own artistic creation and then participate in a cooking class inspired by those masterpieces, promising an expressive and enriching experience.

On the final day, June 12th, thunderstorms are expected as temperatures cool to 28°C, so the itinerary wisely emphasizes indoor venues with flexible options in case of rain. The Soundtrack Picnic at the Starlight Amphitheater offers a delightful combination of movies and live music with an indoor backup plan to ensure the event goes on smoothly. Later that evening, the Tech & Film Fusion Night at the Virtual Reality Theater immerses you in a blend of cutting-edge sci-fi cinema and technology discussions, creating an exciting finale that speaks directly to Yuri’s technological interests and Hiro’s appreciation of movies.

This carefully crafted itinerary balances your interests, weather considerations, and social opportunities, promising a memorable and enjoyable trip to AgentsVille.

## CONGRATULATIONS! 🎉🥳👏

You have successfully planned a stellar vacation to AgentsVille! Your AI travel agent has demonstrated advanced reasoning techniques, including role-based prompting, chain-of-thought reasoning, ReAct prompting, and feedback loops

Give yourself a pat on the back for completing this project and completing this course!